In [1]:
const axios = require('axios');

## HTTP URL
By the HTTP protocol, the target URL consists of host and resource path.
In oneM2M HTTP Binding, there needs the slash('/') handling to build the URL with the host address.
For example, when the host is 'http://localhost:7579' like Mobius default seetting and the target oneM2M resource is my Registrar's <CSEBase> resource e.g. 'Mobius', we need to put the slash in between. For conveninence for implementation here, we put that slash in the end of the base_url so the URL to reach becomes 'http://localhost:7579/Mobius'.
    
## HTTP Method

CRUD operations get mapped to POST, GET, PUT, DELETE HTTP methods. N(Notify) also gets mapped to POST method. Note that in case of CREATE, extra parameter is carried in the HTTP so it can be distinguished with a NOTIFY request.
    
## HTTP Headers
Some of the oneM2M (request and response) parameters mapped to HTTP headers. There are three oneM2M extension headers:
- X-M2M-RI: Request Identifier
- X-M2M-RVI: Request Version Indicator
- X-M2M-Origin: Originator ID
    
Also the accept header should be added.

First request we're going to send out is <CSEBase> ressource retrieval.

In [2]:
const base_url = 'http://localhost:7579/';
let resource_id = 'Mobius';
const options = {
    method: 'get',
    url: base_url + resource_id,
    headers: {
        'X-M2M-RI': 'req123',
        'X-M2M-RVI': '2',
        'X-M2M-Origin': 'SAE1',
        'Accept': 'applicaiton/json'
    }
}

In [3]:
const response = {};

## HTTP Status Codes

oneM2M defines RSC (Response Status Code) in the Core Protocol (TS-0004) and each RSC gets mapped to the HTTP status code. Refer to the HTTP Binding specification for more information (TS-0009).


Some successful responses: 
- on2M2M RSC 2001 (CREATED) => HTTP 201 (created)
- oneM2M RSC 2000 (OK), 2002 (DELETED), 2004 (UPDATED) => HTTP 200 (ok)

Some unsuccessful responses:
- on2M2M RSC 4103 (ORIGINATOR_HAS_NO_PRIVILEGE) => HTTP 403 (forbidden)
- oneM2M RSC 4004 (NOT_FOUND) => HTTP 404 (not found)
- oneM2M RSC 5106 (ALREADY_EXISTS) => HTTP 409 (conflict) 

For more error code mappings, refer to oneM2M 

## oneM2M request in HTTP

Send a request after modifying several parameters and check the response. In case of succesful response, we'll see the response code with returned body which the Primitive Content (pc)  parameter. For the unsuccessful ones, we'll see the debug message from the platform with the error status code.

After you change the parameters below, you can come back here and fire out that request (CRUD) to check the response.

In [61]:
axios(options)
    .then(resp => {
        console.log('status code: ' + resp.status);
        console.log('primitive content: ' + JSON.stringify(resp.data));
    })
    .catch(error => {
        console.log('HTTP status code: ' + error.response.status);
        console.log('oneM2M error message: ' + error.response.data['m2m:dbg']);
});

Promise { <pending> }

HTTP status code: 404
oneM2M error message: resource does not exist (get_target_url)


## Try a create request

Let's try a Create request now, which has HTTP POST method. 
For a CREATE request, the 'Content-Type' request header with 'ty' (Resource Type) shall be added. E.g. 'application/json; ty=3'.

Well-known 'ty' values are:
- 2 (AE)
- 3 (container)
- 4 (contentInstance)
- 23 (subscription)

In [9]:
options.method = 'post';
options.headers['content-type'] = 'application/json; ty=3';

'application/json; ty=3'

### Primitive Content parameter setting

For a CREATE or UPDATE request, the resource instance shall be carried in 'pc' (Primitive Content) parameter which is mapped to HTTP request-body. Once you finish the setting, you can go up again to send the request.

In [10]:
options.data = {"m2m:cnt": {"rn": "cnt_test"}};

{ 'm2m:cnt': { rn: 'cnt_test' } }

## Try a discovery request

A discovery request is a sort of RETRIEVE requests by the oneM2M specification. but add Filter Criteria (fc) parameter having different filter conditions. Before we add 'fc', let's modify the common parameters already set.

In [34]:
options.method = 'get';
delete options.headers['content-type'];
delete options.data;

true

Let's check the request options we made so far.

In [41]:
options

{
  url: 'http://localhost:7579/Mobius',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2'
  },
  method: 'get',
  params: { fu: 1, ty: 3, rn: 'cnt_test' }
}

### Add Filter Criteria for discovery

First condition to add is the indicator that we use 'fc' as discovery.
- fu (filterUsage): 1 (discovery)

And let's discover resource instances which is in *container* resource type
- ty (resourceType): 3 (container)

Also let's search for the container we created by its name
- rn (reosurceName) 

Those are included in the 'fc' as sub-elements like this in JSON:
```
{
    "fc" : {
        "fu": 1,
        "ty": 3,
        "rn": "cnt_test"
    }
}
```

In HTTP binding (TS-0009), each 'fc' elements are carried in HTTP query-string in flat like '?fu=1&ty=3&rn=cnt_test'.
To include these in axios, we can use 'params'

In [52]:
options.params = {
    "fu": 1,
    "ty": 3,
    "rn": "cnt_test"
};
options;

{
  url: 'http://localhost:7579/Mobius/cnt_test',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2',
    'content-type': 'appliation/json'
  },
  method: 'delete',
  params: { fu: 1, ty: 3, rn: 'cnt_test' }
}

## Try an update request

Let's update the <container> resource we created, adding a label (lbl) which has an array of string. It is part of the <container> reosurce so the representation for this is:
```
{
    "m2m:cnt": {
        "lbl": ["keti"]
    }
}
```
    
And note that we use HTTP PUT method with 'Content-Type' header without 'ty' to carry the HTTP body.

In [46]:
options.url = base_url + 'Mobius/cnt_test';
options.method = 'put';
options.headers['content-type'] = 'appliation/json';
options.data = {
    "m2m:cnt": {
        "lbl": ["keti"]
    }
};
options;

{
  url: 'http://localhost:7579/Mobius/cnt_test',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2',
    'content-type': 'appliation/json'
  },
  method: 'put',
  params: { fu: 1, ty: 3, rn: 'cnt_test' },
  data: { 'm2m:cnt': { lbl: [Array] } }
}

Don't forget to remove 'fc' elements for discovery.

In [54]:
delete options.params;
options;

{
  url: 'http://localhost:7579/Mobius/cnt_test',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2',
    'content-type': 'appliation/json'
  },
  method: 'delete'
}

## Try a delete request

Finally let's practice a remove request by removing the <container> resource. We'll use the HTTP delete method and send no payload.

In [59]:
options.method = 'delete';
delete options.headers['content-type'];
delete options.data;
options;

{
  url: 'http://localhost:7579/Mobius/cnt_test',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2'
  },
  method: 'delete'
}

To confirm, let's try to retireve the resource and see what we get in the response.

In [60]:
options.method = 'get';
options;

{
  url: 'http://localhost:7579/Mobius/cnt_test',
  headers: {
    'X-M2M-RI': 'req123',
    'X-M2M-Origin': 'SAE1',
    Accept: 'applicaiton/json',
    'x-m2m-rvi': '2'
  },
  method: 'get'
}